<a href="https://colab.research.google.com/github/Piat0046/PractiseProject/blob/master/StylegGAN2_ADA_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA 모델 학습하기

이 노트북은 다음 노트북을 합쳐서 만들고 한글 설명을 추가했습니다.

- https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb
- https://colab.research.google.com/github/Hephyrius/Stylegan2-Ada-Google-Colab-Starter-Notebook/blob/main/Stylegan2_Ada_Colab_Starter.ipynb

StyleGAN2-ADA 모델은 텐서플로우 1 버전에서만 작동합니다. 아래 셀을 실행해서 1 버전을 사용할 수 있도록 합니다.

In [2]:
pip install numpy==1.19.5

     |████████████████████████████████| 14.8 MB 8.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


현재 구글에서 어떤 GPU를 할당받았는지 알아봅니다. 노트북 위의 메뉴에서 Runtime > Change runtime type 으로 가서 GPU가 선택되었는지 확인하세요.

In [2]:
!nvidia-smi

Sat Mar 19 11:24:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

내 구글 계정의 드라이브를 이 노트북과 연동시킵니다. 앞으로는 구글 드라이브의 파일을 노트북에서 가져다쓰거나, 노트북에서 생성된 파일들을 내 드라이브로 저장할 수 있습니다.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Clone StyleGAN2-ADA Repo

StyleGAN2-ADA 관련된 코드를 노트북으로 불러와서 다운로드받습니다.

In [7]:
# Download the code
%cd /content/
!git clone https://github.com/dvschultz/stylegan2-ada.git
!mkdir downloads
!mkdir datasets

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 364, done.
remote: Total 364 (delta 0), reused 0 (delta 0), pack-reused 364
Receiving objects: 100% (364/364), 56.17 MiB | 34.06 MiB/s, done.
Resolving deltas: 100% (197/197), done.


## Dataset

이번 노트북에서 학습에 사용할 데이터는 포켓몬 이미지 데이터입니다. 다음 링크에서 다운로드받을 수 있습니다:

https://www.kaggle.com/kvpratama/pokemon-images-dataset

다운로드받은 파일은 각자의 구글 드라이브 폴더에 미리 업로드시켜주세요. 업로드한 후에는, 이 노트북에서 데이터를 불러와서 학습에 사용할 수 있습니다. 여기서는 포켓몬 데이터셋 (256x256)을 사용하지만, 다른 데이터셋으로 학습도 가능합니다.

데이터셋의 이름을 zip파일 이름으로 바꿔주세요:

In [27]:
dataset_name = "download1"

노트북으로 데이터셋 압축파일을 불러와서 압축을 풉니다. 왼쪽 Files에서 확인할 수 있습니다.

In [30]:
import zipfile
path = "/content/drive/MyDrive/Colab Notebooks/"
dataset = dataset_name + ".zip"
local_path = "/content/man/"
file_name = path + dataset
with zipfile.ZipFile(file_name, 'r') as zip:
   #zip.printdir()
   print('Extracting all the files now...') 
   zip.extractall(local_path) 
   print('Done!')

Extracting all the files now...
Done!


In [29]:
!ls

aydao_flesh_digressions.py  Dockerfile	  LICENSE.txt	style_mixing.py
blend_models.py		    docs	  metrics	tonni
calc_metrics.py		    ffhq_dataset  notebooks	training
dataset_tool.py		    generate.py   projector.py	train.py
dnnlib			    grid_vid.py   README.md	utils


불러온 이미지 데이터셋 jpg 파일들을 텐서플로에서 사용할 수 있는 데이터셋 포맷(tfrecords)으로 바꿔줍니다. 이 변환과정이 데이터셋의 크기에 따라 굉장히 오래 걸릴 수 있습니다. 이미지 파일들이 저장되어있는 경로를 잘 설정해줘야합니다.

In [59]:
%cd /content/

#update this to the path to your image folder
dataset_path = "/content/stylegan2-ada/tonni/aligned"

#you don't need to edit anything here
%cd /content/stylegan2-ada
!python dataset_tool.py create_from_images /content/datasets/{dataset_name} {dataset_path}

/content
/content/stylegan2-ada
Loading images from "/content/stylegan2-ada/tonni/aligned"
Creating dataset "/content/datasets/download1"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 1550 images.


In [74]:
!python dataset_tool.py -help

usage: dataset_tool.py [-h]
                       {info,display,extract,compare,create_mnist,create_mnistrgb,create_cifar10,create_cifar100,create_svhn,create_lsun,create_lsun_wide,create_celeba,create_from_images,create_from_images_raw,create_from_image_folders,create_from_image_folders_raw,create_from_images_with_labels,create_from_hdf5,convert_to_hdf5,hdf5_from_images,unpack,pack,extract_brecahad_crops}
                       ...
dataset_tool.py: error: argument -h/--help: ignored explicit argument 'elp'


## Training

학습할 때 우리가 조절할 수 있는 다양한 패러미터들이 있습니다. 다음 셀을 실행하면 도움말을 확인할 수 있습니다.

In [66]:
%cd /content/stylegan2-ada
!python train.py --help

/content/stylegan2-ada
usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with L

아래 셀을 실행하면 학습이 시작됩니다. **학습이 진행되는 동안에는, 이 노트북 창을 열어두어야 합니다.** 노트북 창을 닫거나 인터넷 연결이 끊기면, 콜랩 노트북의 실행이 중단됩니다. 또, 너무 오랜시간동안 실행되고 있는 노트북은 구글에서 자동으로 차단합니다. 연결이 끊긴 경우에는 이 노트북을 처음 셀부터 다시 실행해야합니다. 연결이 끊긴 후에 이어서 학습을 하고 싶은 경우에는 `resume_from` 경로를 최근의 `pkl` 파일 경로로 바꿔주세요.

학습이 진행되는 동안 구글 드라이브 폴더를 확인해보세요. 중간 과정의 이미지 결과물과 모델을 자동으로 저장합니다.



In [57]:
%cd ..

/content/stylegan2-ada


In [76]:
# output directory
output_dir = '/content/drive/MyDrive/Colab\ Notebooks/results/' + dataset_name + "/"

# config
config = "auto"

# gamma
gamma = 1

#how often should the model generate samples and a .pkl file
snapshot_count = 2

#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False

#metrics? 
metric_list = None
#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq512"

# make sure there is no space in the resume path. if there is any, use a backslash character to escape
# resume_from = '/content/drive/MyDrive/Colab\ Notebooks/results/pokemon-256/00001-pokemon-256-mirror-auto1-gamma1-resumecustom/network-snapshot-000036.pkl'

# seed

#don't edit this unless you know what you're doing :)
!python train.py --outdir={output_dir} \
                 --cfg={config} \
                 --snap={snapshot_count} \
                 --data=/content/stylegan2-ada/download1 \
                 --mirror={mirrored} --mirrory={mirroredY} \
                 --gamma={gamma} \
                 --metrics={metric_list} \
                 --resume={resume_from} \
                 --seed=900

tcmalloc: large alloc 4294967296 bytes == 0x55702c596000 @  0x7fd08865f001 0x7fd08586254f 0x7fd0858b2b58 0x7fd0858b6b17 0x7fd085955203 0x557023af6424 0x557023af6120 0x557023b6ab80 0x557023b6566e 0x557023af836c 0x557023b397b9 0x557023b366d4 0x557023af6c29 0x557023b6ae61 0x557023b6502f 0x557023a36e2b 0x557023b67633 0x557023b6502f 0x557023a36e2b 0x557023b67633 0x557023b6566e 0x557023a36e2b 0x557023b67633 0x557023af79da 0x557023b65eae 0x557023b6502f 0x557023b64d43 0x557023c2f302 0x557023c2f67d 0x557023c2f526 0x557023c071d3
tcmalloc: large alloc 4294967296 bytes == 0x55712c596000 @  0x7fd08865d1e7 0x7fd08586246e 0x7fd0858b2c7b 0x7fd0858b335f 0x7fd085955103 0x557023af6424 0x557023af6120 0x557023b6ab80 0x557023b6502f 0x557023af7aba 0x557023b66cd4 0x557023b6502f 0x557023af7aba 0x557023b66cd4 0x557023b6502f 0x557023af7aba 0x557023b66cd4 0x557023af79da 0x557023b65eae 0x557023b6502f 0x557023af7aba 0x557023b6a2c0 0x557023b6502f 0x557023af7aba 0x557023b66cd4 0x557023b6566e 0x557023af836c 0x557023b3

In [82]:
!git init
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/Piat0046/stylegan2-ada-.git
#!git push -u origin main

Reinitialized existing Git repository in /content/stylegan2-ada/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@4e2e8063f160.(none)')


In [80]:
!git remote add origin https://github.com/Piat0046/stylegan2-ada-.git
!git branch -M main
!git push -u origin main

fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
